### 1. Configuration and load

In [2]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset loading with error handling
try:
    df = pd.read_csv('../data/raw/customer_support_tickets.csv')
    print("Dataset loaded successfully.")
    print(f'Dataset dimensions: {df.shape[0]} rows and {df.shape[1]} columns')
except FileNotFoundError:
    print("Error: The dataset file was not found.")

Dataset loaded successfully.
Dataset dimensions: 8469 rows and 17 columns


### 2. Data exploration and cleaning

In [5]:
# Display first few rows of the dataset
df.head()

# Basic information about the dataset
df.info()

# Summary statistics
df.describe(include='all')

# Check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic